In [1]:
import pickle
import pandas as pd
import numpy as np
import mudata as mu
import scanpy as sc

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [20]:
path_eReg_direct = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/eRegulon_direct.tsv"
path_eReg_extended = None
path_data = "/cellar/users/aklie/data/datasets/neurips2021_small/small.h5mu"
path_r2g = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/r2g.csv"
path_grn = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/grn.csv"
path_out = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/scenic+.h5mu"

# Load data

In [21]:
# Read in mudata
mdata = mu.read_h5mu(path_data)
mdata

/cellar/users/aklie/opt/miniconda3/envs/test_scenicplus_dev/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_scenicplus_dev/lib/python3.11/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


MuData object with n_obs × n_vars = 123 × 8704
  obs:	'celltype', 'batch', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker'
  2 modalities
    atac:	123 x 8192
      obs:	'pseudotime_order'
      obsm:	'X_lsi', 'X_umap'
      layers:	'counts'
    rna:	123 x 512
      obs:	'pseudotime_order'
      var:	'gene_id'
      obsm:	'X_pca', 'X_umap'
      layers:	'counts'

In [22]:
# Read in eReg direct
egrn = pd.read_csv(path_eReg_direct, sep="\t")
egrn

,Region,Gene,importance_R2G,rho_R2G,importance_x_rho,importance_x_abs_rho,TF,is_extended,eRegulon_name,Gene_signature_name,Region_signature_name,importance_TF2G,regulation,rho_TF2G,triplet_rank
0,chr14:67532787-67533656,VTI1B,0.224823,0.071502,0.016075,0.016075,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.628654,1,0.184340,412
1,chr17:19337794-19338695,EPN2,0.228122,0.261887,0.059742,0.059742,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),3.061339,1,0.267726,400
2,chr7:116210854-116211582,TES,0.579148,0.111869,0.064789,0.064789,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.730566,1,0.177302,229
3,chr4:83535680-83536550,HELQ,0.439058,0.242433,0.106442,0.106442,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.999118,1,0.201019,219
4,chr19:55442053-55442884,UBE2S,0.129482,0.094119,0.012187,0.012187,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),3.211630,1,0.318025,308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,chr19:13164309-13165130,NFIX,0.079568,-0.109065,-0.008678,0.008678,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.664515,-1,-0.057061,276
435,chr22:30609014-30609830,OSBP2,0.144728,-0.396260,-0.057350,0.057350,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),3.117202,-1,-0.171935,239
436,chr17:19289796-19290699,EPN2,0.219785,-0.301227,-0.066205,0.066205,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.703683,-1,-0.065152,237
437,chr20:57398027-57398870,RBM38,0.138520,-0.188692,-0.026138,0.026138,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.951855,-1,-0.059178,273


In [23]:
# Read in eReg extended and concat if necessary
if path_eReg_extended is not None:
    eReg_extended = pd.read_csv(path_eReg_extended, sep="\t")
    egrn = pd.concat([egrn, eReg_extended], axis=0, ignore_index=True)
egrn

,Region,Gene,importance_R2G,rho_R2G,importance_x_rho,importance_x_abs_rho,TF,is_extended,eRegulon_name,Gene_signature_name,Region_signature_name,importance_TF2G,regulation,rho_TF2G,triplet_rank
0,chr14:67532787-67533656,VTI1B,0.224823,0.071502,0.016075,0.016075,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.628654,1,0.184340,412
1,chr17:19337794-19338695,EPN2,0.228122,0.261887,0.059742,0.059742,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),3.061339,1,0.267726,400
2,chr7:116210854-116211582,TES,0.579148,0.111869,0.064789,0.064789,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.730566,1,0.177302,229
3,chr4:83535680-83536550,HELQ,0.439058,0.242433,0.106442,0.106442,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),1.999118,1,0.201019,219
4,chr19:55442053-55442884,UBE2S,0.129482,0.094119,0.012187,0.012187,E2F2,False,E2F2_direct_+/+,E2F2_direct_+/+_(29g),E2F2_direct_+/+_(33r),3.211630,1,0.318025,308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,chr19:13164309-13165130,NFIX,0.079568,-0.109065,-0.008678,0.008678,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.664515,-1,-0.057061,276
435,chr22:30609014-30609830,OSBP2,0.144728,-0.396260,-0.057350,0.057350,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),3.117202,-1,-0.171935,239
436,chr17:19289796-19290699,EPN2,0.219785,-0.301227,-0.066205,0.066205,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.703683,-1,-0.065152,237
437,chr20:57398027-57398870,RBM38,0.138520,-0.188692,-0.026138,0.026138,ERG,False,ERG_direct_-/-,ERG_direct_-/-_(10g),ERG_direct_-/-_(17r),2.951855,-1,-0.059178,273


# tf2r

In [3]:
cistromes_direct = sc.read_h5ad("/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/cistromes_direct.h5ad")
cistromes_direct

AnnData object with n_obs × n_vars = 7421 × 391
    var: 'motifs', 'is_extended'

In [18]:
# Replace the first "-" with ":"


Index(['chr9-101486622-101487347', 'chr15-72237155-72238047',
       'chr6-2124074-2124931', 'chr6-116668283-116669167',
       'chr1-40315014-40315892', 'chr6-3126139-3127040',
       'chr19-3721149-3722057', 'chr1-230149847-230150765',
       'chr3-107598889-107599782', 'chr22-30442071-30442994',
       ...
       'chr21-34201750-34202512', 'chr12-1932013-1932926',
       'chr16-2003561-2004463', 'chr4-56544314-56545210',
       'chr19-1087460-1088358', 'chr17-59440496-59441397',
       'chr6-44096421-44097314', 'chr2-218322752-218323595',
       'chr18-79617538-79618409', 'chr12-68393077-68394014'],
      dtype='object', length=8192)

In [20]:
np.all(cistromes_direct.obs.index.isin([x.replace("-", ":", 1) for x in mdata.mod["atac"].var_names]))

True

In [21]:
cistromes_direct.var

,motifs,is_extended
AFF4,hdpi__AFF4,False
ALX3,metacluster_161.7,False
ALX4,metacluster_161.26,False
ARID3A,metacluster_159.3,False
ATF1,"transfac_pro__M01861,metacluster_111.3",False
...,...,...
ZNF90,kznf__ZNF90_Imbeault2017_OM_RCADE,False
ZNF91,"transfac_pro__M06233,transfac_pro__M06259",False
ZSCAN10,metacluster_55.3,False
ZSCAN22,metacluster_151.1,False


In [22]:
import h5py

In [25]:
path_ctx = "/cellar/users/aklie/data/datasets/neurips2021_small/analysis/scenic+/2024_05_08/ctx_results.hdf5"

In [26]:
file = h5py.File(path_ctx)

In [35]:
file["DARs_Erythroblast/cistromes/region_set"].keys()

<KeysViewHDF5 ['ANHX_extended_(349r)', 'ARID3A_(240r)', 'ARID3A_extended_(325r)', 'ASCL1_extended_(443r)', 'ATF3_extended_(196r)', 'BCLAF1_(37r)', 'BCLAF1_extended_(70r)', 'BHLHE40_extended_(443r)', 'BHLHE41_extended_(443r)', 'BPTF_extended_(37r)', 'BRF1_extended_(30r)', 'CCNT2_(225r)', 'CCNT2_extended_(469r)', 'CDX2_(25r)', 'CDX2_extended_(25r)', 'CEBPA_extended_(99r)', 'CEBPB_extended_(99r)', 'CEBPD_extended_(99r)', 'CEBPE_extended_(99r)', 'CEBPG_extended_(99r)', 'CHURC1_extended_(286r)', 'CLOCK_extended_(49r)', 'CNOT3_(341r)', 'CNOT3_extended_(341r)', 'CREB1_(196r)', 'CREB1_extended_(196r)', 'CTCFL_(240r)', 'CTCFL_extended_(325r)', 'CTCF_(413r)', 'CTCF_extended_(424r)', 'DBP_extended_(179r)', 'DBX1_(18r)', 'DBX1_extended_(18r)', 'DBX2_extended_(18r)', 'DLX2_extended_(69r)', 'DLX3_extended_(69r)', 'DRGX_extended_(49r)', 'E2F1_extended_(354r)', 'E2F2_(213r)', 'E2F2_extended_(213r)', 'E2F4_extended_(486r)', 'E2F6_(221r)', 'E2F6_extended_(723r)', 'E2F7_extended_(391r)', 'E2F8_(249r)', '

# r2g

In [41]:
#
r2g = eReg_direct[["Region", "Gene", "importance_R2G"]]
r2g.columns = ["cre", "gene", "score"]
r2g["pval"] = np.nan
r2g["cluster"] = "global"
r2g

/tmp/ipykernel_845201/1043688769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r2g["pval"] = np.nan
/tmp/ipykernel_845201/1043688769.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r2g["cluster"] = "global"


,cre,gene,score,pval,cluster
0,chr14:67532787-67533656,VTI1B,0.224823,NaN,global
1,chr17:19337794-19338695,EPN2,0.228122,NaN,global
2,chr7:116210854-116211582,TES,0.579148,NaN,global
3,chr4:83535680-83536550,HELQ,0.439058,NaN,global
4,chr19:55442053-55442884,UBE2S,0.129482,NaN,global
...,...,...,...,...,...
434,chr19:13164309-13165130,NFIX,0.079568,NaN,global
435,chr22:30609014-30609830,OSBP2,0.144728,NaN,global
436,chr17:19289796-19290699,EPN2,0.219785,NaN,global
437,chr20:57398027-57398870,RBM38,0.138520,NaN,global


In [44]:
mdata.uns["r2g"] = r2g
r2g.to_csv(path_r2g, index=False)

# grn

In [47]:
grn = eReg_direct[["TF", "Gene", "importance_TF2G"]]
grn.columns = ["tf", "gene", "score"]
grn["pval"] = np.nan
grn["cluster"] = "global"
grn

/tmp/ipykernel_845201/2097319374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grn["pval"] = np.nan
/tmp/ipykernel_845201/2097319374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grn["cluster"] = "global"


,tf,gene,score,pval,cluster
0,E2F2,VTI1B,1.628654,NaN,global
1,E2F2,EPN2,3.061339,NaN,global
2,E2F2,TES,1.730566,NaN,global
3,E2F2,HELQ,1.999118,NaN,global
4,E2F2,UBE2S,3.211630,NaN,global
...,...,...,...,...,...
434,ERG,NFIX,2.664515,NaN,global
435,ERG,OSBP2,3.117202,NaN,global
436,ERG,EPN2,2.703683,NaN,global
437,ERG,RBM38,2.951855,NaN,global


In [48]:
mdata.uns["grn"] = grn
grn.to_csv(path_grn, index=False)

# DONE!

---